In [1]:
# Python ≥3.5 is required
import sys
assert sys.version_info >= (3, 5)

# Scikit-Learn ≥0.20 is required
import sklearn
assert sklearn.__version__ >= "0.20"

# TensorFlow ≥2.0-preview is required
import tensorflow as tf
from tensorflow import keras
assert tf.__version__ >= "2.0"

# Common imports
import numpy as np
import os

# to make this notebook's output stable across runs
np.random.seed(42)

# To plot pretty figures
%matplotlib inline
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rc('axes', labelsize=14)
mpl.rc('xtick', labelsize=12)
mpl.rc('ytick', labelsize=12)

import tensorflow as tf
from tensorflow import keras

In [2]:
keras.layers.Dense(10, activation="relu", kernel_initializer="he_normal")

In [3]:
he_avg_init = keras.initializers.VarianceScaling(scale=2., mode="fan_avg",
                                                distribution="uniform")
keras.layers.Dense(10, activation="sigmoid", kernel_initializer=he_avg_init)

In [4]:
leaky_relu = keras.layers.LeakyReLU(alpha=0.2)
layer = keras.layers.Dense(10, activation=leaky_relu,
                          kernel_initializer="he_normal")

In [5]:
layer = keras.layers.Dense(10, activation="selu",
                          kernel_initializer="lecun_normal")

In [6]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),    
    keras.layers.Dense(500, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

W1012 17:15:08.338925 140279272294208 deprecation.py:323] From /home/joschi/anaconda3/envs/tfgpu/lib/python3.7/site-packages/tensorflow/python/keras/backend.py:4149: add_dispatch_support.<locals>.wrapper (from tensorflow.python.ops.array_ops) is deprecated and will be removed in a future version.
Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


In [7]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
flatten (Flatten)            (None, 784)               0         
_________________________________________________________________
batch_normalization (BatchNo (None, 784)               3136      
_________________________________________________________________
dense_4 (Dense)              (None, 500)               392500    
_________________________________________________________________
batch_normalization_1 (Batch (None, 500)               2000      
_________________________________________________________________
dense_5 (Dense)              (None, 300)               150300    
_________________________________________________________________
batch_normalization_2 (Batch (None, 300)               1200      
_________________________________________________________________
dense_6 (Dense)              (None, 100)               3

In [8]:
[(var.name, var.trainable) for var in model.layers[1].variables]

[('batch_normalization/gamma:0', True),
 ('batch_normalization/beta:0', True),
 ('batch_normalization/moving_mean:0', False),
 ('batch_normalization/moving_variance:0', False)]

In [9]:
model.layers[1].updates

[<tf.Operation 'cond/Identity' type=Identity>,
 <tf.Operation 'cond_1/Identity' type=Identity>]

In [10]:
optimizer = keras.optimizers.Adam()
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=optimizer,
              metrics=["accuracy"])

In [11]:
fashion_mnist = keras.datasets.fashion_mnist
(X_train_full, y_train_full), (X_test, y_test) = fashion_mnist.load_data()

X_valid, X_train = X_train_full[:5000] / 255., X_train_full[5000:] / 255.
y_valid, y_train = y_train_full[:5000], y_train_full[5000:]
X_test = X_test / 255.

In [12]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=10,
                                                 restore_best_weights=True)
history = model.fit(X_train, y_train, epochs=100,
                   validation_data=(X_valid, y_valid),
                   callbacks=[early_stopping_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 10s 191us/sample - loss: 0.4796 - accuracy: 0.8273 - val_loss: 0.3482 - val_accuracy: 0.8718
Epoch 2/100
55000/55000 [==============================] - 9s 164us/sample - loss: 0.3733 - accuracy: 0.8638 - val_loss: 0.3266 - val_accuracy: 0.8762
Epoch 3/100
55000/55000 [==============================] - 9s 164us/sample - loss: 0.3340 - accuracy: 0.8773 - val_loss: 0.3185 - val_accuracy: 0.8802
Epoch 4/100
55000/55000 [==============================] - 9s 163us/sample - loss: 0.3076 - accuracy: 0.8861 - val_loss: 0.3009 - val_accuracy: 0.8924
Epoch 5/100
55000/55000 [==============================] - 9s 167us/sample - loss: 0.2880 - accuracy: 0.8928 - val_loss: 0.2923 - val_accuracy: 0.8918
Epoch 6/100
55000/55000 [==============================] - 9s 164us/sample - loss: 0.2669 - accuracy: 0.8999 - val_loss: 0.3091 - val_accuracy: 0.8920
Epoch 7/100
55000/55000 [===================

In [13]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 75us/sample - loss: 0.3114 - accuracy: 0.8935


In [14]:
model_A = keras.models.load_model("my_model_A.h5")
model_B_on_A = keras.models.Sequential(model_A.layers[:-1])
model_B_on_A.add(keras.layers.Dense(1, activation="sigmoid"))

OSError: SavedModel file does not exist at: my_model_A.h5/{saved_model.pbtxt|saved_model.pb}

In [ ]:
model_A_clone = keras.models.clone_model(model_A)
model_A_clone.set_weights(model_A.get_weights())

In [ ]:
for layer in model_B_on_A.layers[:-1]:
    layer.trainable = False
model_B_on_A.compile(loss="binary_crossentropy", optimizer="sgd",
                    metrics="accuracy")

In [ ]:
optimizer = keras.optimizers.SGD(lr=0.01, decay=1e-4)

In [15]:
def exponential_decay_fn(epoch):
    return 0.01 * 0.1**(epoch / 20)

In [16]:
def exponantial_decay(lr0, s):
    def exponantial_decay_fn(epoch):
        return lr0 * 0.1**(epoch / s)
    return exponantial_decay_fn

In [17]:
lr_scheduler = keras.callbacks.LearningRateScheduler(exponential_decay_fn)
history = model.fit(X_train, y_train, 
                    epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping_cb, lr_scheduler])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 9s 166us/sample - loss: 0.4228 - accuracy: 0.8471 - val_loss: 0.3781 - val_accuracy: 0.8616
Epoch 2/100
55000/55000 [==============================] - 9s 165us/sample - loss: 0.3611 - accuracy: 0.8666 - val_loss: 0.3554 - val_accuracy: 0.8728
Epoch 3/100
55000/55000 [==============================] - 9s 164us/sample - loss: 0.3297 - accuracy: 0.8780 - val_loss: 0.3551 - val_accuracy: 0.8700
Epoch 4/100
55000/55000 [==============================] - 9s 167us/sample - loss: 0.3030 - accuracy: 0.8868 - val_loss: 0.3379 - val_accuracy: 0.8766
Epoch 5/100
55000/55000 [==============================] - 9s 166us/sample - loss: 0.2832 - accuracy: 0.8940 - val_loss: 0.3142 - val_accuracy: 0.8864
Epoch 6/100
55000/55000 [==============================] - 9s 167us/sample - loss: 0.2597 - accuracy: 0.9022 - val_loss: 0.3200 - val_accuracy: 0.8820
Epoch 7/100
55000/55000 [====================

In [18]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 71us/sample - loss: 0.3235 - accuracy: 0.8872


In [19]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.BatchNormalization(),    
    keras.layers.Dense(500, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(300, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(100, activation="selu", kernel_initializer="he_normal"),
    keras.layers.BatchNormalization(),
    keras.layers.Dense(10, activation="softmax")
])

In [20]:
optimizer = keras.optimizers.Adam()
model.compile(loss="sparse_categorical_crossentropy", 
              optimizer=optimizer,
              metrics=["accuracy"])

In [21]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)
history = model.fit(X_train, y_train, 
                    epochs=100,
                    validation_data=(X_valid, y_valid),
                    callbacks=[early_stopping_cb, lr_scheduler])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 10s 183us/sample - loss: 0.4834 - accuracy: 0.8257 - val_loss: 0.3731 - val_accuracy: 0.8668
Epoch 2/100
55000/55000 [==============================] - 9s 168us/sample - loss: 0.3783 - accuracy: 0.8611 - val_loss: 0.3338 - val_accuracy: 0.8802
Epoch 3/100
55000/55000 [==============================] - 9s 164us/sample - loss: 0.3350 - accuracy: 0.8759 - val_loss: 0.3071 - val_accuracy: 0.8882
Epoch 4/100
55000/55000 [==============================] - 9s 168us/sample - loss: 0.3086 - accuracy: 0.8862 - val_loss: 0.3003 - val_accuracy: 0.8890
Epoch 5/100
55000/55000 [==============================] - 9s 166us/sample - loss: 0.2837 - accuracy: 0.8934 - val_loss: 0.3234 - val_accuracy: 0.8830
Epoch 6/100
55000/55000 [==============================] - 9s 165us/sample - loss: 0.2645 - accuracy: 0.9016 - val_loss: 0.2902 - val_accuracy: 0.8944
Epoch 7/100
55000/55000 [===================

In [22]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 1s 76us/sample - loss: 0.3197 - accuracy: 0.8930


In [23]:
s = 20 * len(X_train) // 32
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.Adam(learning_rate)

In [24]:
layer = keras.layers.Dense(100, activation="elu",
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(0.01))

In [25]:
from functools import partial

RegularizedDense = partial(keras.layers.Dense,
                          activation="selu",
                          kernel_initializer="he_normal",
                          kernel_regularizer=keras.regularizers.l2(0.01))

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    RegularizedDense(500),
    RegularizedDense(300),
    RegularizedDense(100),
    RegularizedDense(10, activation="softmax", 
                     kernel_initializer="glorot_uniform")
])

In [26]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

In [27]:
history = model.fit(X_train, y_train,
                   validation_data=(X_valid, y_valid),
                   epochs=100,
                   callbacks=[early_stopping_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 5s 92us/sample - loss: 2.8658 - accuracy: 0.7750 - val_loss: 0.8761 - val_accuracy: 0.8128
Epoch 2/100
55000/55000 [==============================] - 5s 85us/sample - loss: 0.9082 - accuracy: 0.7919 - val_loss: 0.8718 - val_accuracy: 0.8016
Epoch 3/100
55000/55000 [==============================] - 5s 86us/sample - loss: 0.8767 - accuracy: 0.8004 - val_loss: 0.8829 - val_accuracy: 0.7988
Epoch 4/100
55000/55000 [==============================] - 5s 85us/sample - loss: 0.8718 - accuracy: 0.8033 - val_loss: 0.8341 - val_accuracy: 0.8200
Epoch 5/100
55000/55000 [==============================] - 5s 87us/sample - loss: 0.8700 - accuracy: 0.8036 - val_loss: 0.8887 - val_accuracy: 0.7982
Epoch 6/100
55000/55000 [==============================] - 5s 85us/sample - loss: 0.8619 - accuracy: 0.8080 - val_loss: 0.8167 - val_accuracy: 0.8250
Epoch 7/100
55000/55000 [==========================

In [28]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 45us/sample - loss: 0.8363 - accuracy: 0.8149


In [29]:

model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(500, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform"),
])

In [30]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer="adam",
             metrics=["accuracy"])

In [31]:
history = model.fit(X_train, y_train,
                   epochs=100,
                   validation_data=(X_valid, y_valid),
                   callbacks=[early_stopping_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 4s 76us/sample - loss: 0.4889 - accuracy: 0.8212 - val_loss: 0.4183 - val_accuracy: 0.8498
Epoch 2/100
55000/55000 [==============================] - 4s 70us/sample - loss: 0.3805 - accuracy: 0.8591 - val_loss: 0.3478 - val_accuracy: 0.8786
Epoch 3/100
55000/55000 [==============================] - 4s 72us/sample - loss: 0.3406 - accuracy: 0.8719 - val_loss: 0.3615 - val_accuracy: 0.8650
Epoch 4/100
55000/55000 [==============================] - 4s 71us/sample - loss: 0.3203 - accuracy: 0.8797 - val_loss: 0.3548 - val_accuracy: 0.8648
Epoch 5/100
55000/55000 [==============================] - 4s 72us/sample - loss: 0.3001 - accuracy: 0.8859 - val_loss: 0.3459 - val_accuracy: 0.8760
Epoch 6/100
55000/55000 [==============================] - 4s 70us/sample - loss: 0.2853 - accuracy: 0.8922 - val_loss: 0.3396 - val_accuracy: 0.8752
Epoch 7/100
55000/55000 [==========================

In [32]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 33us/sample - loss: 0.3443 - accuracy: 0.8859


In [33]:
early_stopping_cb = keras.callbacks.EarlyStopping(patience=15,
                                                 restore_best_weights=True)
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(500, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.5),
    keras.layers.Dense(300, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.3),
    keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal"),
    keras.layers.Dropout(rate=0.2),
    keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform"),
])

In [34]:
s = 20 * len(X_train) // 32 # number of steps in 20 epochs (batch size = 32)
learning_rate = keras.optimizers.schedules.ExponentialDecay(0.01, s, 0.1)
optimizer = keras.optimizers.SGD(learning_rate)

In [35]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.Adam(learning_rate=5e-4),
             metrics=["accuracy"])

In [36]:
history = model.fit(X_train, y_train,
                   epochs=100,
                   validation_data=(X_valid, y_valid),
                   callbacks=[early_stopping_cb])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 7s 123us/sample - loss: 0.7214 - accuracy: 0.7420 - val_loss: 0.4442 - val_accuracy: 0.8366
Epoch 2/100
55000/55000 [==============================] - 6s 116us/sample - loss: 0.5416 - accuracy: 0.7999 - val_loss: 0.3864 - val_accuracy: 0.8570
Epoch 3/100
55000/55000 [==============================] - 6s 117us/sample - loss: 0.4939 - accuracy: 0.8174 - val_loss: 0.3820 - val_accuracy: 0.8576
Epoch 4/100
55000/55000 [==============================] - 6s 116us/sample - loss: 0.4633 - accuracy: 0.8276 - val_loss: 0.3670 - val_accuracy: 0.8654
Epoch 5/100
55000/55000 [==============================] - 6s 117us/sample - loss: 0.4464 - accuracy: 0.8345 - val_loss: 0.3637 - val_accuracy: 0.8666
Epoch 6/100
55000/55000 [==============================] - 6s 117us/sample - loss: 0.4277 - accuracy: 0.8389 - val_loss: 0.3530 - val_accuracy: 0.8642
Epoch 7/100
55000/55000 [====================

55000/55000 [==============================] - 6s 117us/sample - loss: 0.3013 - accuracy: 0.8860 - val_loss: 0.2835 - val_accuracy: 0.8984
Epoch 55/100
55000/55000 [==============================] - 6s 118us/sample - loss: 0.3010 - accuracy: 0.8867 - val_loss: 0.2827 - val_accuracy: 0.8952
Epoch 56/100
55000/55000 [==============================] - 6s 116us/sample - loss: 0.2980 - accuracy: 0.8884 - val_loss: 0.2777 - val_accuracy: 0.9008
Epoch 57/100
55000/55000 [==============================] - 6s 115us/sample - loss: 0.2990 - accuracy: 0.8891 - val_loss: 0.2866 - val_accuracy: 0.8996
Epoch 58/100
55000/55000 [==============================] - 6s 117us/sample - loss: 0.2983 - accuracy: 0.8889 - val_loss: 0.2815 - val_accuracy: 0.9014
Epoch 59/100
55000/55000 [==============================] - 6s 117us/sample - loss: 0.2947 - accuracy: 0.8883 - val_loss: 0.2825 - val_accuracy: 0.9008
Epoch 60/100
55000/55000 [==============================] - 6s 117us/sample - loss: 0.2958 - accuracy

In [37]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 40us/sample - loss: 0.3036 - accuracy: 0.8902


In [38]:
y_probas = np.stack([model(X_test, training=True)
                       for sample in range(100)])

y_proba = y_probas.mean(axis=0)

In [39]:
np.round(model.predict(X_test[:1]), 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.07, 0.  , 0.93]],
      dtype=float32)

In [40]:
np.round(y_probas[:, :1], 2)

array([[[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.24, 0.  , 0.76]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.99]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.34, 0.  , 0.65]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.69, 0.  , 0.3 ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.99]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.01, 0.  , 0.12, 0.  , 0.87]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.04, 0.  , 0.94]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.39, 0.  , 0.61]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.02, 0.  , 0.28, 0.  , 0.7 ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.19, 0.  , 0.81]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 1.  ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.08, 0.  , 0.72, 0.  , 0.2 ]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.07, 0.  , 0.93]],

       [[0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.  , 0.34, 0.  , 0

In [41]:
np.round(y_proba[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.05, 0.  , 0.2 , 0.  , 0.76]],
      dtype=float32)

In [42]:
y_std = y_probas.std(axis=0)
np.round(y_std[:1], 2)

array([[0.  , 0.  , 0.  , 0.  , 0.  , 0.12, 0.  , 0.2 , 0.  , 0.22]],
      dtype=float32)

In [43]:
y_pred = np.argmax(y_proba, axis=1)
accuracy = np.sum(y_pred == y_test) / len(y_test)
accuracy

0.8929

In [44]:
class MCDropout(keras.layers.Dropout):
    def call(self, inputs):
        return super().call(inputs, training=True)

In [45]:
keras.layers.Dense(100, activation="elu", kernel_initializer="he_normal",
                  kernel_constraint=keras.constraints.max_norm(1.))

In [46]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.Dense(500, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform"),
])

In [47]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.Nadam(),
             metrics=["accuracy"])

In [48]:
lr_scheduler = keras.callbacks.ReduceLROnPlateau(factor=0.5, patience=5)

In [49]:
history = model.fit(X_train, y_train,
                   epochs=100,
                   validation_data=(X_valid, y_valid),
                   callbacks=[early_stopping_cb, lr_scheduler])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 7s 131us/sample - loss: 0.5140 - accuracy: 0.8161 - val_loss: 0.3742 - val_accuracy: 0.8668
Epoch 2/100
55000/55000 [==============================] - 7s 120us/sample - loss: 0.3886 - accuracy: 0.8577 - val_loss: 0.3668 - val_accuracy: 0.8618
Epoch 3/100
55000/55000 [==============================] - 7s 121us/sample - loss: 0.3539 - accuracy: 0.8694 - val_loss: 0.3511 - val_accuracy: 0.8704
Epoch 4/100
55000/55000 [==============================] - 7s 122us/sample - loss: 0.3308 - accuracy: 0.8779 - val_loss: 0.3840 - val_accuracy: 0.8678
Epoch 5/100
55000/55000 [==============================] - 7s 122us/sample - loss: 0.3134 - accuracy: 0.8830 - val_loss: 0.3193 - val_accuracy: 0.8858
Epoch 6/100
55000/55000 [==============================] - 7s 122us/sample - loss: 0.3004 - accuracy: 0.8876 - val_loss: 0.3197 - val_accuracy: 0.8898
Epoch 7/100
55000/55000 [====================

In [50]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 36us/sample - loss: 0.3351 - accuracy: 0.8917


In [51]:
model = keras.models.Sequential([
    keras.layers.Flatten(input_shape=[28, 28]),
    keras.layers.AlphaDropout(rate=0.1),
    keras.layers.Dense(500, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.3),
    keras.layers.Dense(300, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.2),
    keras.layers.Dense(100, activation="selu", kernel_initializer="lecun_normal"),
    keras.layers.AlphaDropout(rate=0.1),
    keras.layers.Dense(10, activation="softmax", kernel_initializer="glorot_uniform"),
])

In [52]:
model.compile(loss="sparse_categorical_crossentropy",
             optimizer=keras.optimizers.Nadam(),
             metrics=["accuracy"])

In [53]:
history = model.fit(X_train, y_train,
                   epochs=100,
                   validation_data=(X_valid, y_valid),
                   callbacks=[early_stopping_cb, lr_scheduler])

Train on 55000 samples, validate on 5000 samples
Epoch 1/100
55000/55000 [==============================] - 12s 214us/sample - loss: 0.8786 - accuracy: 0.6698 - val_loss: 0.7569 - val_accuracy: 0.7676
Epoch 2/100
55000/55000 [==============================] - 11s 201us/sample - loss: 0.7258 - accuracy: 0.7285 - val_loss: 0.8173 - val_accuracy: 0.7730
Epoch 3/100
55000/55000 [==============================] - 11s 200us/sample - loss: 0.6839 - accuracy: 0.7431 - val_loss: 0.7621 - val_accuracy: 0.7872
Epoch 4/100
55000/55000 [==============================] - 11s 201us/sample - loss: 0.6619 - accuracy: 0.7511 - val_loss: 0.6520 - val_accuracy: 0.8092
Epoch 5/100
55000/55000 [==============================] - 11s 200us/sample - loss: 0.6478 - accuracy: 0.7566 - val_loss: 0.7159 - val_accuracy: 0.7988
Epoch 6/100
55000/55000 [==============================] - 11s 200us/sample - loss: 0.6315 - accuracy: 0.7633 - val_loss: 0.5826 - val_accuracy: 0.8220
Epoch 7/100
55000/55000 [==============

In [54]:
evaluate = model.evaluate(X_test, y_test)

10000/10000 [==============================] - 0s 45us/sample - loss: 0.5545 - accuracy: 0.8407


## Exercises

### 1. Is it okay to initialize all the weights to the same value as long as that value is selected randomly using He initialization?

No, because 

### 2. Is it okay to initialize the bias terms to 0?

Yes, because

### 3. Name three advantages of the SELU activation function over ReLU.
1. differentiable at 0
2. non-zero gradients for negative inputs
3. self-normalising as long as the net only consists of dense layers and all layers use SELU as activation function and the following conditions are met:
  3.1 input features must be standardized (mean 0 and standard deviation 1)
  3.2 hidden layer's weight must be initialized using LeCun normal initialization
  3.3 network's architecture must be sequential

### 4. In which cases would you want to use each of the following activation functions: SELU, leaky ReLU (and its variants), ReLU, tanh, logistic, and softmax?
SELU: 

leaky ReLU:

ReLU:

tanh:

logistic: 

softmax: 

### 5. What may happen if you set the momentum hyperparameter too close to 1 (e.g., 0.99999) when using an SGD optimizer?


### 6. Name three ways you can produce a sparse model.
1. strong l1 regularization
2. set very tiny values to 0
3. apply Dual Averaging

### 7. Does dropout slow down training? Does it slow down inference (i.e., making predictions on new instances)? What are about MC dropout?



### 8. Deep Learning

### 9. Transfer Learning

### 10. Pretraining on an auxiliary task